In [5]:
import polars as pl
from statsforecast import StatsForecast
from statsforecast.models import AutoETS, OptimizedTheta

# Create sample dataframe with columns `unique_id`, `ds`, and `y`.
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m4-hourly.parquet")
    .filter(pl.col("unique_id").is_in(["H1", "H2", "H3"]))
    .collect()
)

# Define models
season_length = 24
models = [
    OptimizedTheta(season_length=season_length, decomposition_type="additive"),
    AutoETS(season_length=season_length),
]
sf = StatsForecast(models=models, freq=1, n_jobs=-1)

# Compute the Kaboudan metric in the `ts` namespace
res = df.pts.kaboudan(sf, block_size=200, backtesting_start=0.5, n_folds=10, agg=False)
res

unique_id,OptimizedTheta,AutoETS
str,f64,f64
"""H1""",0.523165,0.130512
"""H2""",0.489781,0.028746
"""H3""",0.45061,0.129111
